In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kreeshrajani/human-stress-prediction

human-stress-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
from zipfile import ZipFile
dataset = '/content/human-stress-prediction.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [ ]:
dataset=pd.read_csv('/content/Stress.csv')
dataset.head()

,subreddit,post_id,sentence_range,text,label,confidence,social_timestamp
0,ptsd,8601tu,"(15, 20)","He said he had not felt that way before, sugge...",1,0.8,1521614353
1,assistance,8lbrx9,"(0, 5)","Hey there r/assistance, Not sure if this is th...",0,1.0,1527009817
2,ptsd,9ch1zh,"(15, 20)",My mom then hit me with the newspaper and it s...,1,0.8,1535935605
3,relationships,7rorpp,"[5, 10]","until i met my new boyfriend, he is amazing, h...",1,0.6,1516429555
4,survivorsofabuse,9p2gbc,"[0, 5]",October is Domestic Violence Awareness Month a...,1,0.8,1539809005


In [ ]:
dataset.shape

(2838, 7)

In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2838 entries, 0 to 2837
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   subreddit         2838 non-null   object 
 1   post_id           2838 non-null   object 
 2   sentence_range    2838 non-null   object 
 3   text              2838 non-null   object 
 4   label             2838 non-null   int64  
 5   confidence        2838 non-null   float64
 6   social_timestamp  2838 non-null   int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 155.3+ KB


In [ ]:
dataset['subreddit'].unique()

array(['ptsd', 'assistance', 'relationships', 'survivorsofabuse',
       'domesticviolence', 'anxiety', 'homeless', 'stress',
       'almosthomeless', 'food_pantry'], dtype=object)

In [ ]:
dataset.isnull().sum()

subreddit           0
post_id             0
sentence_range      0
text                0
label               0
confidence          0
social_timestamp    0
dtype: int64

In [ ]:
dataset.replace({'subreddit':
 {
    'ptsd':0,
    'assistance':1,
    'relationships':2,
    'survivorsofabuse':3,
    'domesticviolence':4,
    'anxiety':5,
    'homeless':6,
    'stress':7,
    'almosthomeless':8,
    'food_pantry':9
}},inplace=True)

In [119]:
x=dataset['text']
y=dataset['label']

In [120]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.1,random_state=2)

In [123]:
from sklearn.feature_extraction.text import TfidfVectorizer
extractor=TfidfVectorizer(min_df=1,lowercase=1,stop_words='english')
x_train_feature=extractor.fit_transform(x_train)
x_test_feature=extractor.transform(x_test)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py:558: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(


In [ ]:
print(x_train_feature)

In [125]:
from sklearn.svm import SVC
model=SVC(kernel='linear')
model.fit(x_train_feature,y_train)

SVC(kernel='linear')

In [126]:
y_pred=model.predict(x_train_feature)
print(y_pred)

[1 1 1 ... 1 1 1]


In [127]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_train,y_pred)
print(accuracy)

0.9553641346906813


In [132]:
dataset['text'][1]

"Hey there r/assistance, Not sure if this is the right place to post this.. but here goes =) I'm currently a student intern at Sandia National Labs and working on a survey to help improve our marketing outreach efforts at the many schools we recruit at around the country. We're looking for current undergrad/grad STEM students so if you're a STEM student or know STEM students, I would greatly appreciate if you can help take or pass along this short survey. As a thank you, everyone who helps take the survey will be entered in to a drawing for chance to win one of three $50 Amazon gcs."

In [133]:
input_text=["Hey there r/assistance, Not sure if this is the right place to post this.. but here goes =) I'm currently a student intern at Sandia National Labs and working on a survey to help improve our marketing outreach efforts at the many schools we recruit at around the country. We're looking for current undergrad/grad STEM students so if you're a STEM student or know STEM students, I would greatly appreciate if you can help take or pass along this short survey. As a thank you, everyone who helps take the survey will be entered in to a drawing for chance to win one of three $50 Amazon gcs."]
input_extraction=extractor.transform(input_text)
prediction=model.predict(input_extraction)
print(prediction)

[0]


In [134]:
y_test_pred=model.predict(x_test_feature)
print(y_test_pred)

[0 1 1 0 1 0 0 0 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 0 0
 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 0
 0 1 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0 1 1 1 1
 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 0 0 0 0 1
 1 1 0 0 0 0 1 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 1 1 1
 0 1 0 1 1 0 1 0 1 1 1 1 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 1 0
 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 1 0 0
 1 1 1 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 1 0 1 0 0 0 1]


In [135]:
y_test_accuracy=accuracy_score(y_test_pred,y_test)
print(y_test_accuracy)

0.7183098591549296


FOR THE USER.........................

In [136]:
input_text=["Hey there r/assistance, Not sure if this is the right place to post this.. but here goes =) I'm currently a student intern at Sandia National Labs and working on a survey to help improve our marketing outreach efforts at the many schools we recruit at around the country. We're looking for current undergrad/grad STEM students so if you're a STEM student or know STEM students, I would greatly appreciate if you can help take or pass along this short survey. As a thank you, everyone who helps take the survey will be entered in to a drawing for chance to win one of three $50 Amazon gcs."]
input_extraction=extractor.transform(input_text)
prediction=model.predict(input_extraction)
if(prediction==1):
  print('The person is having a stress')
else:
  print('The person is not having a stress he is good')

The person is not having a stress he is good
